In [6]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 17.26905120678083,
	"longitude": 82.41842505284906,
	"start_date": "2023-08-06",
	"end_date": "2024-08-04",
	"daily": "weather_code"
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]

daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe.to_csv('ANV.csv',index=False)
print("Done")



In [7]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Initialize cached session with retries
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define a function to fetch weather data
def fetch_weather_data(latitude, longitude, start_date, end_date):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "daily": "weather_code"
    }
    responses = openmeteo.weather_api(url, params=params)
    return responses[0]

# Define a list of locations with their details
locations = [
    {"filename": "slo.csv", "latitude": 17.04547377114114, "longitude": 82.16911532925812},
    {"filename": "ndd.csv", "latitude": 16.899419189510155, "longitude": 81.67491520018432},
    {"filename": "bza.csv", "latitude": 16.51896009123098, "longitude": 80.62001983769811},
    {"filename": "wl.csv", "latitude": 17.973429605430617, "longitude": 79.60582236630121},
    {"filename": "kzj.csv", "latitude": 17.974915531368676, "longitude": 79.51123988184008},
    {"filename": "sc.csv", "latitude": 17.434002801443427,  "longitude": 78.5016622492297},
    {"filename": "hyb.csv", "latitude": 17.392655762857583, "longitude":78.46903534062486},
    # Add more locations as needed
]

# Loop through each location to fetch and save weather data
for location in locations:
    filename = location["filename"]
    latitude = location["latitude"]
    longitude = location["longitude"]
    
    # Fetch weather data
    response = fetch_weather_data(latitude, longitude, "2023-08-06", "2024-08-04")
    
    # Extract daily weather codes
    daily = response.Daily()
    daily_weather_code = daily.Variables(0).ValuesAsNumpy()
    
    # Prepare data for DataFrame
    daily_data = {
        "date": pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s"),
            end=pd.to_datetime(daily.TimeEnd(), unit="s"),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left"
        )
    }
    daily_data["weather_code"] = daily_weather_code
    
    # Create DataFrame and save to CSV
    daily_dataframe = pd.DataFrame(data=daily_data)
    daily_dataframe.to_csv(filename, index=False)
    print(f"Saved weather data for {filename}")

print("All locations processed.")


Saved weather data for slo.csv
Saved weather data for ndd.csv
Saved weather data for bza.csv
Saved weather data for wl.csv
Saved weather data for kzj.csv
Saved weather data for sc.csv
Saved weather data for hyb.csv
All locations processed.
